 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Read whale_returns.csv
whale_returns_filepath = Path("Resources/whale_returns.csv")
whale_returns_df = pd.read_csv(whale_returns_filepath, index_col=["Date"], infer_datetime_format=True, parse_dates=True)

In [ ]:
# Count nulls
whale_returns_df.isnull().sum()

In [ ]:
# Drop nulls
whale_returns_df = whale_returns_df.dropna()

# Verify nulls are removed
whale_returns_df.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_filepath = Path("Resources/algo_returns.csv")
algo_returns_df = pd.read_csv(algo_returns_filepath, index_col=["Date"], infer_datetime_format=True, parse_dates=True)

In [ ]:
# Count nulls
algo_returns_df.isnull().sum()

In [ ]:
# Drop nulls
algo_returns_df = algo_returns_df.dropna()

# Verify nulls are removed
algo_returns_df.isnull().sum()

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_filepath = Path("Resources/sp500_history.csv")
sp500_history_df = pd.read_csv(sp500_history_filepath, index_col=["Date"], infer_datetime_format=True, parse_dates=True)

In [ ]:
# Check Data Types
sp500_history_df.dtypes

In [ ]:
# Fix Data Types
sp500_history_df["Close"] = sp500_history_df["Close"].str.replace("$", "")
sp500_history_df["Close"] = sp500_history_df["Close"].astype(float)

sp500_history_df.dtypes

In [ ]:
# Calculate Daily Returns
sp500_history_df = sp500_history_df.sort_index()
sp500_returns_df = sp500_history_df.pct_change()

In [ ]:
# Drop nulls
sp500_returns_df = sp500_returns_df.dropna()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp500_returns_df = sp500_returns_df.rename(columns={"Close":"S&P 500"})

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combined_returns_df = pd.concat([whale_returns_df, algo_returns_df, sp500_returns_df], axis="columns", join="inner")
combined_returns_df

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_returns_df.plot(figsize=(20,10))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + combined_returns_df).cumprod()

# Plot cumulative returns
cumulative_returns.plot(figsize=(20,10))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk


### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
combined_std_df = combined_returns_df.std()

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate the daily standard deviation of S&P 500
sp500_std = combined_returns_df["S&P 500"].std()
print(f"S&P 500 Standard Deviation: {sp500_std:.6f}")

# Determine which portfolios are riskier than the S&P 500
combined_std_df

# Tiger Global Management LLC and Berkshire Hathaway Inc are riskier than the S&P 500 because it has higher deviation

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_std = combined_returns_df.std() * np.sqrt(252)

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for the S&P 500 using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_21 = combined_returns_df.rolling(window=21).std()

# Plot the rolling standard deviation
rolling_21.plot(figsize=(20,10))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
combined_correlation = combined_returns_df.corr(method="pearson")

# Display the correlation matrix
sns.heatmap(combined_correlation, vmin=-1, vmax=1, annot=True)

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
algo1_covariance = combined_returns_df["Algo 1"].cov(combined_returns_df["S&P 500"])

# Calculate variance of S&P 500
sp500_variance = combined_returns_df["S&P 500"].var()

# Computing beta
algo1_beta = algo1_covariance / sp500_variance

# Plot beta trend
algo1_rolling30_cov = combined_returns_df["Algo 1"].rolling(window=30).cov(combined_returns_df["S&P 500"])
sp500_rolling30_var = combined_returns_df["S&P 500"].rolling(window=30).var()
algo1_rolling30_beta = algo1_rolling30_cov / sp500_rolling30_var
algo1_rolling30_beta.plot(figsize=(20,10))

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = ((combined_returns_df.mean() * 252) - 0.0025) / annualized_std
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [ ]:
# Reading data from 1st stock
goog_filepath = Path("Resources/GOOG.csv")
goog_df = pd.read_csv(goog_filepath, index_col="Date", parse_dates=True, infer_datetime_format=True)
goog_df.head()

In [ ]:
# Reading data from 2nd stock
fb_filepath = Path("Resources/FB.csv")
fb_df = pd.read_csv(fb_filepath, index_col="Date", parse_dates=True, infer_datetime_format=True)
fb_df.head()

In [ ]:
# Reading data from 3rd stock
msft_filepath = Path("Resources/MSFT.csv")
msft_df = pd.read_csv(msft_filepath, index_col="Date", parse_dates=True, infer_datetime_format=True)
msft_df.head()

In [ ]:
# Combine all stocks in a single DataFrame
combined_stocks_df = pd.concat([goog_df, fb_df, msft_df], axis="columns", join="inner")
combined_stocks_df.columns = ["GOOG", "FB", "MSFT"]
combined_stocks_df.head()

In [ ]:
# Reset Date index
combined_stocks_df.sort_index(inplace=True)
combined_stocks_df.head()

In [ ]:
# Reorganize portfolio data by having a column per symbol


In [ ]:
# Calculate daily returns
stocks_daily_returns = combined_stocks_df.pct_change()

# Drop NAs
stocks_daily_returns = stocks_daily_returns.dropna()

# Display sample data
stocks_daily_returns.sample(10)

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
weighted_returns = stocks_daily_returns.dot(weights)

# Display sample data
weighted_returns.sample(10)
# weighted_returns.plot()

# cumu_weighted_returns = (1 + weighted_returns).cumprod()
# cumu_weighted_returns.plot()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame


In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualzied Sharpe Ratios


In [ ]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!